### Probabilistc ERUPT in CATE Setting

Let's suppose we have two treatments for a medical condition, and we want to evaluate their effectiveness for patients segmented by age group. Our goal is to use probabilistic ERUPT to compare different models that estimate CATE.

#### Treatments

- **Treatment A**: Medication A
- **Treatment B**: Medication B

#### Data from Clinical Trials

Here is hypothetical data showing patient outcomes after receiving each treatment, segmented by age group:

| Patient | Age Group | Treatment | Outcome |
|---------|-----------|-----------|---------|
| 1       | Young     | A         | 5       |
| 2       | Young     | A         | 3       |
| 3       | Young     | B         | 2       |
| 4       | Old       | B         | 4       |
| 5       | Old       | A         | 6       |
| 6       | Old       | B         | 5       |

#### Model Predictions

Two different models provide CATE estimates (impact) and uncertainties (standard deviations) for each treatment, segmented by age group.

**Model 1 Estimates:**

| Age Group | Treatment | Estimated Impact | Std Deviation |
|-----------|-----------|------------------|---------------|
| Young     | A         | 4.5              | 1.0           |
| Young     | B         | 3.5              | 0.5           |
| Old       | A         | 5.5              | 0.8           |
| Old       | B         | 4.0              | 0.6           |

**Model 2 Estimates:**

| Age Group | Treatment | Estimated Impact | Std Deviation |
|-----------|-----------|------------------|---------------|
| Young     | A         | 4.0              | 0.8           |
| Young     | B         | 4.0              | 0.8           |
| Old       | A         | 5.0              | 0.7           |
| Old       | B         | 5.0              | 0.7           |

### Probabilistic ERUPT Process in CATE Setting

Here’s how the process can be adapted to evaluate and compare these models in the CATE setting:

1. **Probabilistic Treatment Selection Based on CATE**:
   - We use Thompson sampling to select treatments probabilistically based on the CATE estimates and their uncertainties.

   For each age group, we sample from the distributions defined by the models' estimates.

   **Example of one iteration for each model:**

   - **Model 1**:
     - **Young Group**: Samples 4.2 for A and 3.8 for B — chooses A.
     - **Old Group**: Samples 5.1 for A and 3.7 for B — chooses A.

   - **Model 2**:
     - **Young Group**: Samples 3.5 for A and 4.5 for B — chooses B.
     - **Old Group**: Samples 4.6 for A and 5.2 for B — chooses B.

2. **Simulate the Selection Process**:
   - Perform this sampling many times to simulate different scenarios.
   - Calculate how often each treatment is chosen in each age group.

3. **Calculate ERUPT Scores**:
   - For each model, after many iterations, calculate the average outcome for the treatments chosen based on the probabilistic CATE estimates.

   Historical outcomes based on the table:

   - **Young Group**: Average outcomes for A: 4, for B: 2.
   - **Old Group**: Average outcomes for A: 6, for B: 4.5.

   **Calculated over many iterations:**

   - **Model 1**: 
     - Young Group: More often chooses A.
     - Old Group: More often chooses A.
     - Average outcome = $(0.7 * 4 + 0.3 * 2)_{Young} + (0.8 * 6 + 0.2 * 4.5)_{Old}$
     - $= (0.7 * 4 + 0.3 * 2) + (0.8 * 6 + 0.2 * 4.5)$
     - $= (2.8 + 0.6) + (4.8 + 0.9) = 3.4 + 5.7 = 9.$

   - **Model 2**: 
     - Young Group: More often chooses B.
     - Old Group: More often chooses B.
     - Average outcome = $(0.4 * 4 + 0.6 * 2)_{Young} + (0.4 * 6 + 0.6 * 4.5)_{Old}$
     - $= (0.4 * 4 + 0.6 * 2) + (0.4 * 6 + 0.6 * 4.5)$
     - $= (1.6 + 1.2) + (2.4 + 2.7) = 2.8 + 5.1 = 7.9$

4. **Compare Models**:
   - Compare these average outcomes. The model with the higher average outcome is considered better at using CATE for treatment selection.

### Conclusion

In this example, **Model 1** appears to perform better in utilizing CATE to probabilistically select more effective treatments, based on the outcomes in each segment. This approach shows how probabilistic ERUPT can be a powerful tool for comparing models in terms of their ability to use CATE estimates effectively under uncertainty.

### Suitable Estimators (Provide unertainty naturally)

Naturally:
- Causal Forest DML: Provides uncertainties naturally from the forest structure.
- DR Ortho Forest: Provides standard errors naturally.
- DML Ortho Forest: Also provides standard errors naturally.


With conditions:
- T-Learner: If the underlying models provide standard deviations.
- X-Learner: If the second-stage models provide standard deviations.
- Forest DR Learner: Can provide standard deviations through the variability of forest predictions.
- Linear DR Learner: Can provide standard errors for predictions.
- Sparse Linear DR Learner: Can provide uncertainties through methods like bootstrapping.
- Linear DML: Provides standard errors if the outcome model is linear.
- Sparse Linear DML: Similar to Linear DML with potential bootstrapping.

In [ ]:
import copy
import logging
import math
from typing import Optional, Dict, Union, Any, List

import numpy as np
import pandas as pd
from sklearn.preprocessing import QuantileTransformer

from econml.cate_interpreter import SingleTreeCateInterpreter  # noqa F401
from dowhy.causal_estimator import CausalEstimate
from dowhy import CausalModel


from causaltune.thirdparty.causalml import metrics
from causaltune.erupt import ERUPT
from causaltune.utils import treatment_values, psw_joint_weights

import dcor


class DummyEstimator:
    def __init__(
        self, cate_estimate: np.ndarray, effect_intervals: Optional[np.ndarray] = None
    ):
        self.cate_estimate = cate_estimate
        self.effect_intervals = effect_intervals

    def const_marginal_effect(self, X):
        return self.cate_estimate


def supported_metrics(problem: str, multivalue: bool, scores_only: bool) -> List[str]:
    if problem == "iv":
        metrics = ["energy_distance"]
        if not scores_only:
            metrics.append("ate")
        return metrics
    elif problem == "backdoor":
        print("backdoor")
        if multivalue:
            # TODO: support other metrics for the multivalue case
            return ["energy_distance", "psw_energy_distance"]
        else:
            metrics = [
                "erupt",
                "norm_erupt",
                "prob_erupt",
                "qini",
                "auc",
                # "r_scorer",
                "energy_distance",
                "psw_energy_distance",
            ]
            if not scores_only:
                metrics.append("ate")
            return metrics


class Scorer:
    def __init__(
        self,
        causal_model: CausalModel,
        propensity_model: Any,
        problem: str,
        multivalue: bool,
    ):
        """
        Contains scoring logic for CausalTune.

        Access methods and attributes via `CausalTune.scorer`.

        """

        self.problem = problem
        self.multivalue = multivalue
        self.causal_model = copy.deepcopy(causal_model)

        self.identified_estimand = causal_model.identify_effect(
            proceed_when_unidentifiable=True
        )

        if problem == "backdoor":
            print(
                "Fitting a Propensity-Weighted scoring estimator to be used in scoring tasks"
            )
            treatment_series = causal_model._data[causal_model._treatment[0]]
            # this will also fit self.propensity_model, which we'll also use in self.erupt
            self.psw_estimator = self.causal_model.estimate_effect(
                self.identified_estimand,
                method_name="backdoor.causaltune.models.MultivaluePSW",
                control_value=0,
                treatment_value=treatment_values(treatment_series, 0),
                target_units="ate",  # condition used for CATE
                confidence_intervals=False,
                method_params={
                    "init_params": {"propensity_model": propensity_model},
                },
            ).estimator

            treatment_name = self.psw_estimator._treatment_name
            if not isinstance(treatment_name, str):
                treatment_name = treatment_name[0]

            # No need to call self.erupt.fit() as propensity model is already fitted
            # self.propensity_model = est.propensity_model
            self.erupt = ERUPT(
                treatment_name=treatment_name,
                propensity_model=self.psw_estimator.estimator.propensity_model,
                X_names=self.psw_estimator._effect_modifier_names
                + self.psw_estimator._observed_common_causes_names,
            )

    def ate(self, df: pd.DataFrame) -> tuple:
        """Calculate the Average Treatment Effect. Provide naive std estimates in single-treatment cases.

        Args:
            df (pandas.DataFrame): input dataframe

        Returns:
            tuple: tuple containing the ATE, standard deviation of the estimate (or None if multi-treatment),
                and sample size (or None if estimate has more than one dimension)

        """

        estimate = self.psw_estimator.estimator.effect(df).mean(axis=0)

        if len(estimate) == 1:
            # for now, let's cheat on the std estimation, take that from the naive ate
            treatment_name = self.causal_model._treatment[0]
            outcome_name = self.causal_model._outcome[0]
            naive_est = Scorer.naive_ate(df[treatment_name], df[outcome_name])
            return estimate[0], naive_est[1], naive_est[2]
        else:
            return estimate, None, None

    def resolve_metric(self, metric: str) -> str:
        """Check if supplied metric is supported. If not, default to 'energy_distance'.

        Args:
            metric (str): evaluation metric

        Returns:
            str: metric/'energy_distance'

        """

        metrics = supported_metrics(self.problem, self.multivalue, scores_only=True)

        if metric not in metrics:
            logging.warning(
                f"Using energy_distance metric as {metric} is not in the list "
                f"of supported metrics for this usecase ({str(metrics)})"
            )
            return "energy_distance"
        else:
            return metric

    def resolve_reported_metrics(
        self, metrics_to_report: Union[List[str], None], scoring_metric: str
    ) -> List[str]:
        """Check if supplied reporting metrics are valid.

        Args:
            metrics_to_report (Union[List[str], None]): list of strings specifying the evaluation metrics to compute.
                Possible options include 'ate', 'erupt', 'norm_erupt', 'qini', 'auc',
                'energy_distance' and 'psw_energy_distance'.
            scoring_metric (str): specified metric

        Returns:
            List[str]: list of valid metrics
        """

        metrics = supported_metrics(self.problem, self.multivalue, scores_only=False)
        if metrics_to_report is None:
            return metrics
        else:
            metrics_to_report = sorted(list(set(metrics_to_report + [scoring_metric])))
            for m in metrics_to_report.copy():
                if m not in metrics:
                    logging.warning(
                        f"Dropping the metric {m} for problem: {self.problem} \
                        : must be one of {metrics}"
                    )
                    metrics_to_report.remove(m)
        return metrics_to_report

    @staticmethod
    def energy_distance_score(
        estimate: CausalEstimate,
        df: pd.DataFrame,
    ) -> float:
        """Calculate energy distance score between treated and controls.
        For theoretical details, see Ramos-Carreño and Torrecilla (2023).

        Args:
            estimate (dowhy.causal_estimator.CausalEstimate): causal estimate to evaluate
            df (pandas.DataFrame): input dataframe

        Returns:
            float: energy distance score

        """

        Y0X, _, split_test_by = Scorer._Y0_X_potential_outcomes(estimate, df)

        YX_1 = Y0X[Y0X[split_test_by] == 1]
        YX_0 = Y0X[Y0X[split_test_by] == 0]
        select_cols = estimate.estimator._effect_modifier_names + ["yhat"]

        energy_distance_score = dcor.energy_distance(
            YX_1[select_cols], YX_0[select_cols]
        )

        return energy_distance_score

    @staticmethod
    def _Y0_X_potential_outcomes(estimate: CausalEstimate, df: pd.DataFrame):
        est = estimate.estimator
        # assert est.identifier_method in ["iv", "backdoor"]
        treatment_name = (
            est._treatment_name
            if isinstance(est._treatment_name, str)
            else est._treatment_name[0]
        )
        df["dy"] = estimate.estimator.effect_tt(df)
        df["yhat"] = df[est._outcome_name] - df["dy"]

        split_test_by = (
            est.estimating_instrument_names[0]
            if est.identifier_method == "iv"
            else treatment_name
        )

        Y0X = copy.deepcopy(df)
        return Y0X, treatment_name, split_test_by

    def psw_energy_distance(
        self,
        estimate: CausalEstimate,
        df: pd.DataFrame,
        normalise_features=False,
    ) -> float:
        """
        Calculate propensity score adjusted energy distance score between treated and controls.

        Features are normalised using the sklearn.preprocessing.QuantileTransformer

        For theoretical details, see Ramos-Carreño and Torrecilla (2023).

        @param estimate (dowhy.causal_estimator.CausalEstimate): causal estimate to evaluate
        @param df (pandas.DataFrame): input dataframe
        @param normalise_features (bool): whether to normalise features with QuantileTransformer

        @return float: propensity-score weighted energy distance score

        """

        Y0X, treatment_name, split_test_by = Scorer._Y0_X_potential_outcomes(
            estimate, df
        )

        Y0X_1 = Y0X[Y0X[split_test_by] == 1]
        Y0X_0 = Y0X[Y0X[split_test_by] == 0]

        YX_1_all_psw = self.psw_estimator.estimator.propensity_model.predict_proba(
            Y0X_1[
                self.causal_model.get_effect_modifiers()
                + self.causal_model.get_common_causes()
            ]
        )
        treatment_series = Y0X_1[treatment_name]

        YX_1_psw = np.zeros(YX_1_all_psw.shape[0])
        for i in treatment_series.unique():
            YX_1_psw[treatment_series == i] = YX_1_all_psw[:, i][treatment_series == i]

        YX_0_psw = self.psw_estimator.estimator.propensity_model.predict_proba(
            Y0X_0[
                self.causal_model.get_effect_modifiers()
                + self.causal_model.get_common_causes()
            ]
        )[:, 0]

        select_cols = estimate.estimator._effect_modifier_names + ["yhat"]
        features = estimate.estimator._effect_modifier_names

        xy_psw = psw_joint_weights(YX_1_psw, YX_0_psw)
        xx_psw = psw_joint_weights(YX_0_psw)
        yy_psw = psw_joint_weights(YX_1_psw)

        xy_mean_weights = np.mean(xy_psw)
        xx_mean_weights = np.mean(xx_psw)
        yy_mean_weights = np.mean(yy_psw)

        if normalise_features:
            qt = QuantileTransformer(n_quantiles=200)
            X_quantiles = qt.fit_transform(Y0X[features])

            Y0X_transformed = pd.DataFrame(
                X_quantiles, columns=features, index=Y0X.index
            )
            Y0X_transformed.loc[:, ["yhat", split_test_by]] = Y0X[
                ["yhat", split_test_by]
            ]

            Y0X_1 = Y0X_transformed[Y0X_transformed[split_test_by] == 1]
            Y0X_0 = Y0X_transformed[Y0X_transformed[split_test_by] == 0]

        exponent = 1
        distance_xy = np.reciprocal(xy_mean_weights) * np.multiply(
            xy_psw,
            dcor.distances.pairwise_distances(
                Y0X_1[select_cols], Y0X_0[select_cols], exponent=exponent
            ),
        )
        distance_yy = np.reciprocal(yy_mean_weights) * np.multiply(
            yy_psw,
            dcor.distances.pairwise_distances(Y0X_1[select_cols], exponent=exponent),
        )
        distance_xx = np.reciprocal(xx_mean_weights) * np.multiply(
            xx_psw,
            dcor.distances.pairwise_distances(Y0X_0[select_cols], exponent=exponent),
        )
        psw_energy_distance = (
            2 * np.mean(distance_xy) - np.mean(distance_xx) - np.mean(distance_yy)
        )
        return psw_energy_distance

    @staticmethod
    def qini_make_score(
        estimate: CausalEstimate, df: pd.DataFrame, cate_estimate: np.ndarray
    ) -> float:
        """Calculate the Qini score, defined as the area between the Qini curves of a model and random.

        Args:
            estimate (dowhy.causal_estimator.CausalEstimate): causal estimate to evaluate
            df (pandas.DataFrame): input dataframe
            cate_estimate (np.ndarray): array with cate estimates

        Returns:
            float: Qini score

        """

        est = estimate.estimator
        new_df = pd.DataFrame()
        new_df["y"] = df[est._outcome_name]
        treatment_name = est._treatment_name
        if not isinstance(treatment_name, str):
            treatment_name = treatment_name[0]
        new_df["w"] = df[treatment_name]
        new_df["model"] = cate_estimate

        qini_score = metrics.qini_score(new_df)

        return qini_score["model"]

    @staticmethod
    def auc_make_score(
        estimate: CausalEstimate, df: pd.DataFrame, cate_estimate: np.ndarray
    ) -> float:
        """Calculate the area under the uplift curve.

        Args:
            estimate (dowhy.causal_estimator.CausalEstimate): causal estimate to evaluate
            df (pandas.DataFrame): input dataframe
            cate_estimate (np.ndarray): array with cate estimates

        Returns:
            float: area under the uplift curve

        """

        est = estimate.estimator
        new_df = pd.DataFrame()
        new_df["y"] = df[est._outcome_name]
        treatment_name = est._treatment_name
        if not isinstance(treatment_name, str):
            treatment_name = treatment_name[0]
        new_df["w"] = df[treatment_name]
        new_df["model"] = cate_estimate

        auc_score = metrics.auuc_score(new_df)

        return auc_score["model"]

    @staticmethod
    def real_qini_make_score(
        estimate: CausalEstimate, df: pd.DataFrame, cate_estimate: np.ndarray
    ) -> float:
        # TODO  To calculate the 'real' qini score for synthetic datasets, to be done

        # est = estimate.estimator
        new_df = pd.DataFrame()

        # new_df['tau'] = [df['y_factual'] - df['y_cfactual']]
        new_df["model"] = cate_estimate

        qini_score = metrics.qini_score(new_df)

        return qini_score["model"]

    @staticmethod
    def r_make_score(
        estimate: CausalEstimate, df: pd.DataFrame, cate_estimate: np.ndarray, r_scorer
    ) -> float:
        """Calculate r_score.
        For details refer to Nie and Wager (2017) and Schuler et al. (2018). Adaption from EconML implementation.

        Args:
            estimate (dowhy.causal_estimator.CausalEstimate): causal estimate to evaluate
            df (pandas.DataFrame): input dataframe
            cate_estimate (np.ndarray): array with cate estimates
            r_scorer: callable object used to compute the R-score

        Returns:
            float: r_score

        """

        # TODO
        return r_scorer.score(cate_estimate)

    @staticmethod
    def naive_ate(treatment: pd.Series, outcome: pd.Series):
        """Calculate simple ATE.

        Args:
            treatment (pandas.Series): series of treatments
            outcome (pandas.Series): series of outcomes

        Returns:
            tuple: tuple of simple ATE, standard deviation, and sample size

        """

        treated = (treatment == 1).sum()

        mean_ = outcome[treatment == 1].mean() - outcome[treatment == 0].mean()
        std1 = outcome[treatment == 1].std() / (math.sqrt(treated) + 1e-3)
        std2 = outcome[treatment == 0].std() / (
            math.sqrt(len(outcome) - treated) + 1e-3
        )
        std_ = math.sqrt(std1 * std1 + std2 * std2)
        return (mean_, std_, len(treatment))

    def group_ate(
        self, df: pd.DataFrame, policy: Union[pd.DataFrame, np.ndarray]
    ) -> pd.DataFrame:
        """Compute the average treatment effect (ATE) for different groups specified by a policy.

        Args:
            df (pandas.DataFrame): input dataframe, should contain columns for the treatment, outcome, and policy
            policy (Union[pd.DataFrame, np.ndarray]): policy column in df or an array of the policy values,
                used to group the data

        Returns:
            pandas.DataFrame: ATE, std, and size per policy

        """

        tmp = {"all": self.ate(df)}
        for p in sorted(list(policy.unique())):
            tmp[p] = self.ate(df[policy == p])

        tmp2 = [
            {"policy": str(p), "mean": m, "std": s, "count": c}
            for p, (m, s, c) in tmp.items()
        ]

        return pd.DataFrame(tmp2)

    def make_scores(
        self,
        estimate: CausalEstimate,
        df: pd.DataFrame,
        metrics_to_report: List[str],
        r_scorer=None,
    ) -> dict:
        """Calculate various performance metrics for a given causal estimate using a given DataFrame.

        Args:
            estimate (dowhy.causal_estimator.CausalEstimate): causal estimate to evaluate
            df (pandas.DataFrame): input dataframe
            metrics_to_report (List[str]): list of strings specifying the evaluation metrics to compute.
                Possible options include 'ate', 'erupt', 'norm_erupt', 'qini', 'auc',
                'energy_distance' and 'psw_energy_distance'.
            r_scorer (Optional): callable object used to compute the R-score, default is None

        Returns:
            dict: dictionary containing the evaluation metrics specified in metrics_to_report.
                The values key in the dictionary contains the input DataFrame with additional columns for
                the propensity scores, the policy, the normalized policy, and the weights, if applicable.
        """

        out = dict()
        df = df.copy().reset_index()

        est = estimate.estimator
        treatment_name = est._treatment_name
        if not isinstance(treatment_name, str):
            treatment_name = treatment_name[0]
        outcome_name = est._outcome_name

        cate_estimate = est.effect(df)

        # TODO: fix this hack with proper treatment of multivalues
        if len(cate_estimate.shape) > 1 and cate_estimate.shape[1] == 1:
            cate_estimate = cate_estimate.reshape(-1)

        # TODO: fix this, currently broken
        # covariates = est._effect_modifier_names
        # Include CATE Interpereter for both IV and CATE models
        # intrp = SingleTreeCateInterpreter(
        #     include_model_uncertainty=False, max_depth=2, min_samples_leaf=10
        # )
        # intrp.interpret(DummyEstimator(cate_estimate), df[covariates])
        # intrp.feature_names = covariates
        # out["intrp"] = intrp

        if self.problem == "backdoor":
            values = df[[treatment_name, outcome_name]]
            simple_ate = self.ate(df)[0]
            if isinstance(simple_ate, float):
                # simple_ate = simple_ate[0]
                # .reset_index(drop=True)
                values[
                    "p"
                ] = self.psw_estimator.estimator.propensity_model.predict_proba(
                    df[
                        self.causal_model.get_effect_modifiers()
                        + self.causal_model.get_common_causes()
                    ]
                )[
                    :, 1
                ]
                values["policy"] = cate_estimate > 0
                values["norm_policy"] = cate_estimate > simple_ate
                values["weights"] = self.erupt.weights(df, lambda x: cate_estimate > 0)
            else:
                pass
                # TODO: what do we do here if multiple treatments?

            if "erupt" in metrics_to_report:
                erupt_score = self.erupt.score(df, df[outcome_name], cate_estimate > 0)
                out["erupt"] = erupt_score

            if "norm_erupt" in metrics_to_report:
                norm_erupt_score = (
                    self.erupt.score(df, df[outcome_name], cate_estimate > simple_ate)
                    - simple_ate * values["norm_policy"].mean()
                )
                out["norm_erupt"] = norm_erupt_score

            if "prob_erupt" in metrics_to_report:
                treatment_effects = pd.Series(cate_estimate, index=df.index)
                treatment_std_devs = pd.Series(cate_estimate.std(), index=df.index)
                prob_erupt_score = self.erupt.probabilistic_erupt_score(df, df[outcome_name], treatment_effects, treatment_std_devs)
                out["prob_erupt"] = prob_erupt_score

            if "qini" in metrics_to_report:
                out["qini"] = Scorer.qini_make_score(estimate, df, cate_estimate)

            if "auc" in metrics_to_report:
                out["auc"] = Scorer.auc_make_score(estimate, df, cate_estimate)

            if r_scorer is not None:
                out["r_score"] = Scorer.r_make_score(
                    estimate, df, cate_estimate, r_scorer
                )

            # values = values.rename(columns={treatment_name: "treated"})
            assert len(values) == len(df), "Index weirdness when adding columns!"
            values = values.copy()
            out["values"] = values

        if "ate" in metrics_to_report:
            out["ate"] = cate_estimate.mean()
            out["ate_std"] = cate_estimate.std()

        if "energy_distance" in metrics_to_report:
            out["energy_distance"] = Scorer.energy_distance_score(estimate, df)

        if "psw_energy_distance" in metrics_to_report:
            out["psw_energy_distance"] = self.psw_energy_distance(
                estimate,
                df,
            )

        del df
        return out

    @staticmethod
    def best_score_by_estimator(
        scores: Dict[str, dict], metric: str
    ) -> Dict[str, dict]:
        """Obtain best score for each estimator.

        Args:
            scores (Dict[str, dict]): CausalTune.scores dictionary
            metric (str): metric of interest

        Returns:
            Dict[str, dict]: dictionary containing best score by estimator

        """

        for k, v in scores.items():
            if "estimator_name" not in v:
                raise ValueError(
                    f"Malformed scores dict, 'estimator_name' field missing in {k}, {v}"
                )

        estimator_names = sorted(
            list(
                set(
                    [
                        v["estimator_name"]
                        for v in scores.values()
                        if "estimator_name" in v
                    ]
                )
            )
        )
        best = {}
        for name in estimator_names:
            est_scores = [
                v
                for v in scores.values()
                if "estimator_name" in v and v["estimator_name"] == name
            ]
            best[name] = (
                min(est_scores, key=lambda x: x[metric])
                if metric in ["energy_distance", "psw_energy_distance"]
                else max(est_scores, key=lambda x: x[metric])
            )

        return best


In [ ]:
from typing import Callable, List, Optional, Union
import copy

import pandas as pd
import numpy as np
import copy
import numpy as np
import pandas as pd
from typing import Callable, List, Optional, Union

# implementation of https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3111957
# we assume treatment takes integer values from 0 to n


class DummyPropensity:
    def __init__(self, p: pd.Series, treatment: pd.Series):
        n_vals = max(treatment) + 1
        out = np.zeros((len(treatment), n_vals))
        for i, pp in enumerate(p.values):
            out[i, treatment.values[i]] = pp
        self.p = out

    def fit(self, *args, **kwargs):
        pass

    def predict_proba(self):
        return self.p


class ERUPT:
    def __init__(
        self,
        treatment_name: str,
        propensity_model,
        X_names: Optional[List[str]] = None,
        clip: float = 0.05,
        remove_tiny: bool = True,
    ):
        self.treatment_name = treatment_name
        self.propensity_model = copy.deepcopy(propensity_model)
        self.X_names = X_names
        self.clip = clip
        self.remove_tiny = remove_tiny

    def fit(self, df: pd.DataFrame):
        if self.X_names is None:
            self.X_names = [c for c in df.columns if c != self.treatment_name]
        self.propensity_model.fit(df[self.X_names], df[self.treatment_name])

    def score(
        self, df: pd.DataFrame, outcome: pd.Series, policy: Callable
    ) -> pd.Series:
        w = self.weights(df, policy)
        return (w * outcome).mean()

    def weights(
        self, df: pd.DataFrame, policy: Union[Callable, np.ndarray, pd.Series]
    ) -> pd.Series:
        W = df[self.treatment_name].astype(int)
        assert all(
            [x >= 0 for x in W.unique()]
        ), "Treatment values must be non-negative integers"

        if callable(policy):
            policy = policy(df).astype(int)
        if isinstance(policy, pd.Series):
            policy = policy.values
        policy = np.array(policy)

        d = pd.Series(index=df.index, data=policy)
        assert all(
            [x >= 0 for x in d.unique()]
        ), "Policy values must be non-negative integers"

        if isinstance(self.propensity_model, DummyPropensity):
            p = self.propensity_model.predict_proba()
        else:
            p = self.propensity_model.predict_proba(df[self.X_names])
        p = np.maximum(p, 1e-4)

        weight = np.zeros(len(df))

        for i in W.unique():
            weight[W == i] = 1 / p[:, i][W == i]

        weight[d != W] = 0.0

        if self.remove_tiny:
            weight[weight > 1 / self.clip] = 0.0
        else:
            weight[weight > 1 / self.clip] = 1 / self.clip

        weight *= len(df) / sum(weight)
        assert not np.isnan(weight.sum()), "NaNs in ERUPT weights"

        return pd.Series(index=df.index, data=weight)

    def probabilistic_erupt_score(
        self, df: pd.DataFrame, outcome: pd.Series, treatment_effects: pd.Series, treatment_std_devs: pd.Series, iterations: int = 1000
    ) -> float:
        unique_treatments = df[self.treatment_name].unique()
        treatment_scores = {treatment: [] for treatment in unique_treatments}

        for _ in range(iterations):
            sampled_effects = {
                treatment: np.random.normal(treatment_effects.loc[treatment], treatment_std_devs.loc[treatment])
                for treatment in unique_treatments
            }
            chosen_treatment = max(sampled_effects, key=sampled_effects.get)
            # Compute weighted outcome
            weights = self.weights(df, lambda x: np.array([chosen_treatment] * len(x)))
            mean_outcome = (weights * outcome).sum() / weights.sum()
            treatment_scores[chosen_treatment].append(mean_outcome)

        average_outcomes = np.mean([np.mean(scores) for scores in treatment_scores.values() if scores])

        return average_outcomes

# Topic Groups with Questions

## (Ridge / LASSO) Regression

3. (2019) Consider the multiple linear regression model without the intercept term,

$$
\mathbf{Y}=\mathbf{X} \boldsymbol{\beta}+\boldsymbol{\varepsilon}
$$

where $\mathbf{Y}=\left(Y_{1}, \ldots, Y_{n}\right)^{T}, \boldsymbol{\beta}=\left(\beta_{1}, \ldots, \beta_{p}\right)^{T}, \boldsymbol{\varepsilon}=\left(\varepsilon_{1}, \ldots, \varepsilon_{n}\right)^{T} \sim N\left(\mathbf{0}, \sigma^{2} \mathbf{I}_{n}\right)$ and $\mathbf{X}=$ $\left(\mathbf{v}_{1}, \ldots, \mathbf{v}_{p}\right) \in \mathbb{R}^{n \times p}$ is the design matrix with $\mathbf{v}_{j}=\left(X_{1 j}, \ldots, X_{n j}\right)^{T}$. The ridge regression estimator solves the following minimisation problem,

$$
\min _{\boldsymbol{\beta} \in \mathbb{R}^{p}}\|\mathbf{Y}-\mathbf{X} \boldsymbol{\beta}\|^{2}+\lambda \sum_{j=1}^{p} \beta_{j}^{2}
$$

where $\lambda$ is a non-negative tuning parameter. In class, we have shown that the solution is given by

$$
\widehat{\boldsymbol{\beta}}_{\text {ridge }}=\left(\mathbf{X}^{T} \mathbf{X}+\lambda \mathbf{I}_{p}\right)^{-1} \mathbf{X}^{T} \mathbf{Y}
$$

Suppose that the columns of $\mathbf{X}$ are mutually orthogonal.

(a) Show that $\widehat{\boldsymbol{\beta}}_{\text {ridge }}$ can be found element by element by solving $p$ separate ridge regression problems as follows,

$$
\min _{\beta_{j} \in \mathbb{R}}\left\|\mathbf{Y}-\mathbf{v}_{j} \beta_{j}\right\|^{2}+\lambda \beta_{j}^{2}, j=1, \ldots, p
$$

(b) Show that the sum of squares explained by the full ridge regression is the sum of those explained in each of the $p$ separate ridge regressions, that is

$$
\left(\mathbf{X} \widehat{\boldsymbol{\beta}}_{\text {ridge }}\right)^{T}\left(\mathbf{X} \widehat{\boldsymbol{\beta}}_{\text {ridge }}\right)=\sum_{j=1}^{p}\left(\mathbf{v}_{j} \widehat{\beta}_{\text {ridge }, \mathrm{j}}\right)^{T}\left(\mathbf{v}_{j} \widehat{\beta}_{\text {ridge }, \mathrm{j}}\right)
$$

where $\widehat{\beta}_{\text {ridge }, j}$ is the $j$-th component of $\widehat{\boldsymbol{\beta}}_{\text {ridge }}$ for $j=1, \ldots, p$.

[10 marks]

(c) Let $\mathbf{v}_{1}^{T} \mathbf{v}_{1}=2$. Find the mean square error (MSE) for $\widehat{\beta}_{\text {ridge, }}$. What is the value of $\lambda$ that minimises $\operatorname{MSE}\left(\widehat{\beta}_{\text {ridge }, 1}\right)$ ?

[9 marks]

2 (d) (2020) For multiple linear regression model without the intercept term,

$$
\mathbf{Y}=\mathbf{X} \boldsymbol{\beta}+\boldsymbol{\varepsilon}
$$

where $\mathbf{Y}=\left(Y_{1}, \ldots, Y_{n}\right)^{T}, \boldsymbol{\beta}=\left(\beta_{1}, \ldots, \beta_{p}\right)^{T}, \boldsymbol{\varepsilon}=\left(\varepsilon_{1}, \ldots, \varepsilon_{n}\right)^{T} \sim N\left(\mathbf{0}, \sigma^{2} \mathbf{I}_{n}\right)$ and $\mathbf{X} \in \mathbb{R}^{n \times p}$. Consider the following optimization problem:

$$
\min _{\boldsymbol{\beta} \in \mathbb{R}^{p}}\|\mathbf{Y}-\mathbf{X} \boldsymbol{\beta}\|^{2}+\lambda \sum_{j=1}^{p}\left[\alpha \beta_{j}^{2}+(1-\alpha)\left|\beta_{j}\right|\right]
$$

where $\lambda \geq 0$ and $0 \leq \alpha \leq 1$ are two tuning parameters. Show how one can turn the above optimization problem into an instance of the lasso problem, using augmented versions of $\mathbf{X}$ and $\mathbf{Y}$, denoted by $\widetilde{\mathbf{X}}$ and $\widetilde{\mathbf{Y}}$, respectively. What is the resulting lasso problem in terms of $\widetilde{\mathbf{X}}, \widetilde{\mathbf{Y}}$ and the corresponding tuning parameter $\widetilde{\lambda}$ ? [7 marks]

3. (2021) Consider multiple linear regression without the intercept term. Let $\boldsymbol{\beta}=\left(\beta_{1}, \ldots, \beta_{p}\right)^{\mathrm{T}}$ and $\mathbf{x}_{i}=\left(x_{i 1}, \ldots, x_{i p}\right)^{\mathrm{T}}$. Suppose that the training data $\left\{\left(\mathbf{x}_{i}, y_{i}\right)\right\}_{i=1}^{n}$ satisfy

$$
y_{i}=\boldsymbol{\beta}^{\mathrm{T}} \mathbf{x}_{i}+\varepsilon_{i}, \quad i=1, \ldots, n
$$

and the test data $\left\{\left(\tilde{\mathbf{x}}_{i^{\prime}}, \tilde{y}_{i^{\prime}}\right)\right\}_{i^{\prime}=1}^{m}$ satisfy

$$
\tilde{y}_{i^{\prime}}=\boldsymbol{\beta}^{\mathrm{T}} \tilde{\mathbf{x}}_{i^{\prime}}+\tilde{\varepsilon}_{i^{\prime}}, \quad i^{\prime}=1, \ldots, m
$$

where $\tilde{\mathbf{x}}_{i^{\prime}}=\left(\tilde{x}_{i^{\prime} 1}, \ldots, \tilde{x}_{i^{\prime} p}\right)^{\mathrm{T}}$.

(a) An estimator is scale-invariant in the sense that the test set prediction accuracy is unchanged if one rescales $\mathbf{x}_{i}$ and $\tilde{\mathbf{x}}_{i^{\prime}}$ as $c \mathbf{x}_{i}$ and $c \tilde{\mathbf{x}}_{i^{\prime}}$, respectively, for all $i, i^{\prime}$, where $c$ is some nonzero constant. You may use formulas for the least squares and ridge regression estimators of $\beta$ without proof.

i. Justify whether the least squares estimator is scale-invariant or not. [6 marks]

ii. Justify whether the ridge regression estimator with a fixed tuning parameter $\lambda$ is scale-invariant or not.

[6 marks]
(b) Suppose that $\left\{\left(\mathbf{x}_{i}, y_{i}\right)\right\}_{i=1}^{n}$ and $\left\{\left(\tilde{\mathbf{x}}_{i^{\prime}}, \tilde{y}_{i^{\prime}}\right)\right\}_{i^{\prime}=1}^{m}$ are drawn at random from a population. Let $\hat{\boldsymbol{\beta}}$ be the least squares estimator obtained from $\left\{\left(\mathbf{x}_{i}, y_{i}\right)\right\}_{i=1}^{n}$. If $R_{\mathrm{tr}}(\boldsymbol{\beta})=$ $\frac{1}{n} \sum_{i=1}^{n}\left(y_{i}-\boldsymbol{\beta}^{\mathrm{T}} \mathbf{x}_{i}\right)^{2}$ and $R_{\mathrm{te}}(\boldsymbol{\beta})=\frac{1}{m} \sum_{i^{\prime}=1}^{m}\left(\tilde{y}_{i^{\prime}}-\boldsymbol{\beta}^{\mathrm{T}} \tilde{\mathbf{x}}_{i^{\prime}}\right)^{2}$, prove that

$$
\mathbb{E}\left[R_{\mathrm{tr}}(\hat{\boldsymbol{\beta}})\right] \leq \mathbb{E}\left[R_{\mathrm{te}}(\hat{\boldsymbol{\beta}})\right]
$$

where expectations are over all that is random in each expression.

[9 marks]


## Tree Based Methods

#### Classification Trees
4. (2019) This problem is about the construction of classification trees. Each split is typically made in order to minimise the cost function

$$
C(T)=\sum_{m=1}^{|T|} N_{m} Q_{m}
$$

where $T$ is the tree object, $|T|$ is total number of terminal nodes, $N_{m}$ is the number of training data points in the region corresponding to the $m$-th terminal node, and $Q_{m}$ is the node impurity measure of the same region. In class, we have introduced two common measures including "misclassification error" and "Gini index". Consider a two-class classification problem with two inputs and a training dataset illustrated in Figure 2. There are infinitely many possible splits we could make, but we restrict ourselves to split at integer values in the region where the training data is located.

Figure 2: Training dataset with two inputs $X_{1}, X_{2}$ and two classes for $Y$ : "solid circle" and "hollow circle".

(a) Give an optimal classification tree with two split points for minimizing the training misclassification loss.

[4 marks]

(b) If we use the misclassification error as the node impurity measure, what is the resulting classification tree with two split points obtained by recursive binary splitting? Explain the suboptimality of the solution.

[5 marks]

(c) Consider the first split point. If we use the Gini index as the node impurity measure, compute the cost $C(T)$ for the following two candidate splits
i. $R_{1}=\left\{X: X_{1} \leq 1\right\}$ and $R_{2}=\left\{X: X_{1}>1\right\}$,

ii. $R_{1}=\left\{X: X_{1} \leq 2\right\}$ and $R_{2}=\left\{X: X_{1}>2\right\}$.

Among the above two candidate splits in i. and ii., where would we make the first split?

[6 marks]

2. (2021) This problem is on the topic of constructing classification trees. Each split can be made based on the cost function

$$
C(T)=\sum_{m=1}^{|T|} N_{m} Q_{m}
$$

Consider a two-class classification problem with two inputs and a training dataset illustrated in Figure 2. For the first split point, we consider the following three candidate splits:
i. $R_{1}=\left\{X: X_{1} \leq 0.25\right\}$ and $R_{2}=\left\{X: X_{1}>0.25\right\}$,

ii. $R_{1}=\left\{X: X_{1} \leq 0.5\right\}$ and $R_{2}=\left\{X: X_{1}>0.5\right\}$,

iii. $R_{1}=\left\{X: X_{1} \leq 0.75\right\}$ and $R_{2}=\left\{X: X_{1}>0.75\right\}$.

(a) If we use misclassification error rate as the node impurity measure $Q_{m}$, compute $C(T)$ for splits in i. and iii. Among two candidate splits in i. and iii., where would we make the split?

(b) If we use Gini index as the node impurity measure $Q_{m}$, compute the cost $C(T)$ for splits in ii. and iii. Among two candidate splits in ii. and iii., where would we make the split?

[8 marks]


Figure 2: Training dataset with $X_{1}, X_{2}$ and two classes for $Y$ : "red circle" and "blue square".

#### Regression Trees
5. (2021) Consider $y_{i}=f\left(\mathbf{x}_{i}\right)+\varepsilon_{i}$ for $i=1, \ldots, n$, where $\varepsilon_{i}$ 's are i.i.d. with mean zero and variance $\sigma^{2}$, and $\mathbf{x}_{i}=\left(x_{i 1}, \ldots, x_{i p}\right)^{\mathrm{T}}$. Let $\mathbf{x}_{i} \in[0,1]^{p}=[0,1] \times \cdots \times[0,1]$. Divide each $[0,1]$ into intervals with the same length $h>0$. Hence, $[0,1]^{p}$ is divided into $J$ non-overlapping $p$ dimensional regions, $B_{1}, \ldots, B_{J}$. Treat $\mathbf{x}_{i}$ 's as fixed. For $\mathbf{x} \in[0,1]^{p}$, let

$$
\hat{f}(\mathbf{x})=\frac{1}{N(\mathbf{x})} \sum_{i=1}^{n} y_{i} I\left(\mathbf{x}_{i} \in B(\mathbf{x})\right)
$$

where $B(\mathbf{x})$ is the $p$-dimensional region containing $\mathbf{x}$ and $N(\mathbf{x})=\sum_{i=1}^{n} I\left(\mathbf{x}_{i} \in B(\mathbf{x})\right)$ is the number of $\mathbf{x}_{i}{ }^{\prime} \mathrm{s}$ in $B(\mathbf{x})$. Further assume that $\left|f(\mathbf{x})-f\left(\mathbf{x}^{\prime}\right)\right| \leq L\left\|\mathbf{x}-\mathbf{x}^{\prime}\right\|=L\left\{\sum_{j=1}^{p}\left(x_{j}-\right.\right.$ $\left.\left.x_{j}^{\prime}\right)^{2}\right\}^{1 / 2}$ for some positive constant $L$ and all $\mathbf{x}, \mathbf{x}^{\prime} \in[0,1]^{p}$. Prove that

$$
\mathbb{E}\left[(\hat{f}(\mathbf{x})-f(\mathbf{x}))^{2}\right] \leq L^{2} p h^{2}+\frac{\sigma^{2}}{N(\mathbf{x})}
$$

(Hint: consider the variance and bias decomposition of the mean squared error.)

[13 marks]



## Clustering (k-means)

5. (2019) Let $\mathbf{x}_{1}, \ldots, \mathbf{x}_{n}$ be a dataset of $p$-dimensional vectors and $\left\{C_{1}, \ldots, C_{K}\right\}$ be a partition of $\{1, \ldots, n\}$. Let $n_{k}$ be the number of observations in cluster $C_{k}$ for $k=1, \ldots, K$. For each cluster $C_{k}$, define $\overline{\mathbf{x}}_{k}=\frac{1}{n_{k}} \sum_{i \in C_{k}} \mathbf{x}_{i}$ to be the within-cluster mean and $\overline{\mathbf{x}}=\frac{1}{n} \sum_{i=1}^{n} \mathbf{x}_{i}$ to
be the overall mean. Let

$\mathbf{T}=\sum_{k=1}^{K} \sum_{i \in C_{k}}\left(\mathbf{x}_{i}-\overline{\mathbf{x}}\right)\left(\mathbf{x}_{i}-\overline{\mathbf{x}}\right)^{T}$ to be the total deviance to the overall mean,

$\mathbf{W}=\sum_{k=1}^{K} \sum_{i \in C_{k}}\left(\mathbf{x}_{i}-\overline{\mathbf{x}}_{k}\right)\left(\mathbf{x}_{i}-\overline{\mathbf{x}}_{k}\right)^{T}$ to be the within-cluster deviance to the cluster mean,

$\mathbf{B}=\sum_{k=1}^{K} \sum_{i \in C_{k}}\left(\mathbf{x}_{k}-\overline{\mathbf{x}}\right)\left(\mathbf{x}_{k}-\overline{\mathbf{x}}\right)^{T}$ to be the between cluster deviance,

where $\mathbf{T}, \mathbf{W}$ and $\mathbf{B} \in \mathbb{R}^{p \times p}$.
(a) Verify that $\mathbf{T}=\mathbf{W}+\mathbf{B}$.

(b) Explain how the objective function of $K$-means is related to $\mathbf{W}$.

(c) Explain how $\mathbf{T}$ and $\mathbf{B}$ change during the course of the $K$-means algorithm.

[6 marks]

5. (2020) Some of the most commonly-used types of linkage in hierarchical clustering are

- Single linkage: distance between clusters is the minimum distance between any pair of points from two clusters.


(a)
(b)
(c)

Figure 3: For Question 5.

- Complete linkage: distance between clusters is the maximum distance between any pair of points from two clusters.
- Average linkage: distance between clusters is the average distance between any pair of points from two clusters.

(a) Which of the three types of linkage described above would most likely result in clusters most similar to those given by $k$-means?

[3 marks]

(b) Consider the data in Figure 3 (a). What would be the result if we extract 2 clusters from the tree given by hierarchical clustering on this dataset using single linkage? Describe your answer in terms of the labels 1-4 given to the four "clumps" in the data. Do the same for complete linkage and average linkage.

[6 marks]

(c) Which of the three types of linkage (if any) would successfully separate the two "moons" in Figure 3 (b)? What about Figure 3 (c)? Briefly explain your answers.

[6 marks]


## Bayes Classifier

3. (2020) (a) In classification, the loss function we usually want to minimize is the $0 / 1$ loss:

$$
\ell(f(X), Y))=I(f(X) \neq Y)
$$

where $I(\cdot)$ denotes an indicator function, which is one if the statement in $(\cdot)$ is true and 0 otherwise. Choose the label $Y \sim$ Bernoulli(1/2), which is 1 with probability $1 / 2$. If $Y=1, X \sim \operatorname{Bernoulli}(p)$, which is 1 with probability $p$. If $Y=0, X \sim \operatorname{Bernoulli}(q)$. Suppose that $p>q$. What is the Bayes optimal classifier and what is its risk?

[7 marks]
(b) In the following problem, we will consider the effect of using an asymmetric loss function:

$$
\ell_{\alpha, \beta}(f(X), Y)=\alpha I(f(X)=1, Y=0)+\beta I(f(X)=0, Y=1)
$$

Under this loss function, the two types of errors receive different weights, determined by $\alpha, \beta>0$.

i. What is the Bayes optimal classifier?

[7 marks]

ii. Suppose that class $Y=0$ is extremely uncommon, that is $P(Y=0)$ is small. This means $f(x)=1$ for all $x$ will have good risk. We may try to put the two classes on the same footing by considering the risk:

$$
R=P(f(X)=1 \mid Y=0)+P(f(X)=0 \mid Y=1)
$$

Show how $R$ is equivalent to the risk where the loss function is $\ell_{\alpha, \beta}$ with certain choices of $\alpha$ and $\beta$.

[6 marks]


4. (2021) (a) Consider the following binary classification problem, $Y=0$ (class 1 ) or 1 (class 2). Suppose that $X \in[0,1]$ with uniform distribution on $[0,1]$ and

$$
P(Y=1 \mid X=x)=2\left|x-\frac{1}{2}\right|
$$

i. Find the expression for the Bayes classifier.

ii. Calculate the Bayes risk $R^{*}$ for this classification task.

[9 marks]

(b) Consider a binary classification problem, $Y=0$ (Class 1) or 1 (Class 2) with twodimensional predictor variable $\mathbf{X}$. The estimated mean vectors of two classes are $\hat{\boldsymbol{\mu}}_{1}=(-1,-1)^{\mathrm{T}}$ and $\hat{\boldsymbol{\mu}}_{2}=(1,1)^{\mathrm{T}}$, respectively. The estimated prior probabilities $\hat{\pi}_{1}$ and $\hat{\pi}_{2}$ are equal.

i. Applying LDA, we estimate the covariance matrix by $\widehat{\boldsymbol{\Sigma}}=\left(\begin{array}{ll}1 & \rho \\ \rho & 1\end{array}\right)$, where $0 \leq \rho<1$. Find the decision boundary as a function of $\rho$.

[6 marks]

ii. Applying QDA, we estimate the covariance matrices of two classes by $\hat{\boldsymbol{\Sigma}}_{1}=$ $\left(\begin{array}{cc}3 & 0 \\ 0 & 1 / 3\end{array}\right)$ and $\hat{\boldsymbol{\Sigma}}_{2}=\left(\begin{array}{cc}1 / 3 & 0 \\ 0 & 3\end{array}\right)$, respectively. Find the decision rule and draw a sketch of it in a two-dimensional plane.

[9 marks]


## Smoothing Splines

4. (2020) The smoothing spline aims to find $g(\cdot)$ that minimizes the penalized residual sum of squares

$$
\sum_{i=1}^{n}\left(y_{i}-g\left(x_{i}\right)\right)^{2}+\lambda \int\left(g^{(2)}(t)\right)^{2} d t
$$

where $\left\{\left(x_{i}, y_{i}\right)\right\}_{i=1}^{n}$ denote the observed values for the covariate and the response, $\lambda \geq 0$ is a smoothing parameter and $g^{(2)}$ represents the second derivative of $g$. Write $g(x)=$ $\mathbf{b}(x)^{T} \boldsymbol{\theta}$, where $\mathbf{b}(x)=\left(b_{1}(x), \ldots, b_{n}(x)\right)^{T} \in \mathbb{R}^{n}$ is a $n$-dimensional basis function and $\boldsymbol{\theta}=\left(\theta_{1}, \ldots, \theta_{n}\right)^{T} \in \mathbb{R}^{n}$. Let $\mathbf{y}=\left(y_{1}, \ldots, y_{n}\right)^{T}, \mathbf{B} \in \mathbb{R}^{n \times n}$ with its $i$-th row vector given by $\mathbf{b}\left(x_{i}\right) \in \mathbb{R}^{n}$ and $\boldsymbol{\Omega}=\int \mathbf{b}^{(2)}(t) \mathbf{b}^{(2)}(t)^{T} d t \in \mathbb{R}^{n \times n}$ with $\mathbf{b}^{(2)}(x)=\left(b_{1}^{(2)}(x), \ldots, b_{n}^{(2)}(x)\right)^{T}$. It was shown in class that

$$
\widehat{\mathbf{g}}_{\lambda}=\mathbf{S}_{\lambda} \mathbf{y}
$$

where $\mathbf{S}_{\lambda}=\mathbf{B}\left(\mathbf{B}^{T} \mathbf{B}+\lambda \boldsymbol{\Omega}\right)^{-1} \mathbf{B}^{T}$.

(a) Suppose that $\mathbf{B}$ is invertible. Express $\mathbf{S}_{\lambda}=\left(\mathbf{I}_{n}+\lambda \mathbf{K}\right)^{-1}$, where $\mathbf{I}_{n}$ is a $n \times n$ identity matrix. What is the expression for $\mathbf{K}$ ?

[5 marks]

(b) For $\mathbf{K}$, let its eigenvalues be $d_{1}, \ldots, d_{n}$ with the corresponding orthonormal eigenvectors, $\mathbf{u}_{1}, \ldots, \mathbf{u}_{n} \in \mathbb{R}^{n}$. Express $\mathbf{S}_{\lambda}$ in terms of $\lambda$ and $\left\{\left(d_{i}, \mathbf{u}_{i}\right)\right\}_{i=1}^{n}$.

[6 marks]

(c) Discuss how the smoothing spline fitted value, $\widehat{\mathrm{g}}_{\lambda}$, differs from the fitted value by regressing $\mathbf{y}$ on the orthonormal basis $\mathbf{u}_{1}, \ldots, \mathbf{u}_{n}$.

[6 marks]

(d) Express the degrees of freedom $\mathrm{df}_{\lambda}=\operatorname{trace}\left(\mathbf{S}_{\lambda}\right)$ in terms of $\lambda$ and $\left\{d_{i}\right\}_{i=1}^{n}$. [ 6 marks]


## k-Nearest Neighbours

2. (2020) (a) i. Provide a 2-dimensional dataset where 1-nearest neighbour has lower LOOCV error than linear SVM.

ii. Provide a 2-dimensional dataset where 1-nearest neighbour has higher LOOCV error than linear SVM.
(b) Consider the $k$-nearest neighbours approach using Euclidean distance on the dataset shown in Figure 2. What are the Leave-One-Out Cross Validation (LOOCV) errors for the following cases? Briefly justify your answers.

i. 1-nearest neighbour.

ii. 3-nearest neighbours.

Figure 2: For Question 2. (b)

(c) Consider the following binary classification problem. At a data point $x$, the conditional probability of a class $Y=k, k \in\{0,1\}$ is $p_{k}(x)=P(Y=k \mid X=x)$. In terms of $p_{k}(x)$ and $p_{k}\left(x^{\prime}\right)$ when $x^{\prime}$ is the nearest neighbour of $x$, what is the 1-nearest neighbour error at $x$ ?

[3 marks]

## Misclassification Error and ROC Curves

2. (2019) Consider a two-class classification problem, $Y \in\{0,1\}$. A classifier is constructed as $\widehat{G}(X)=I(\widehat{p}(X)>r)$ for some user-chosen threshold $r \in[0,1]$, where $\widehat{p}(X) \in(0,1)$ is the predicted probability for $Y=1$ given the input $X$ and $I(\cdot)$ is the indicator function. The classifier is trained by minimizing the misclassification error of a training dataset $\left\{\left(x_{i}, y_{i}\right)\right\}_{i=1}^{n}$ with size $n=1000$ for $r=0.5$. The number of positive examples are $\sum_{i=1}^{n} I\left(Y_{i}=1\right)=600$. Figure 1 plots a ROC curve for the classifier based on the training dataset, that is a plot of the true positive rate (TPR) vs the false positive rate (FPR) as $r$ varies.

Figure 1: ROC curve for thresholds $r \in[0,1]$.
(a) A point corresponding to a specific value of $r$ is marked with a label $(\mathrm{FPR}=0.66$, $\mathrm{TPR}=0.82$ in Figure 1. Construct a confusion matrix for this value of $r$. [6 marks]

(b) Express the misclassification error rate as a function of FPR and TPR. [6 marks]

(c) It just so happens that this ROC curve is well approximately by the function $\mathrm{TPR}=$ $\sqrt{\mathrm{FPR}}$. Using this approximation, compute the FPR for the threshold value $r=0.5$. [5 marks]




The problem presented involves classifying the performance of a student, either as passing or failing, based on the number of sleeping hours ($X$) and a measure of laziness ($Z$). Here's the breakdown of the problem and the solution provided for part (i):

### Problem Setting:
- The classification rule is $Y = 1$ (pass) if $X + Z \leq 12$, and $Y = 0$ (fail) otherwise.
- The variables $X$ (sleeping hours) and $Z$ (laziness measure) are both uniformly distributed over the interval $[0,10]$, and they are independent of each other.

### Objective for Part (i):
To find the expression for the Bayes classifier with respect to $X$, you need to derive the classifier that minimizes the probability of misclassification based on the value of $X$ alone.

### Solution Explanation:
#### Step 1: Calculate $\eta(x)$
$\eta(x)$ is defined as the conditional probability that $Y = 1$ given $X = x$. This translates mathematically to:
$$
\eta(x) = P(Y = 1 \mid X = x) = P(X + Z \leq 12 \mid X = x) = P(Z \leq 12 - X \mid X = x).
$$

#### Step 2: Considering the Range of $X$ and $Z$
Since $X$ and $Z$ are independent and uniformly distributed on $[0, 10]$, the density of $Z$ given any value of $X = x$ remains uniform over $[0, 10]$. Therefore, the probability of $Z \leq 12 - X$ is straightforward to compute:
- If $12 - X \geq 10$, i.e., $X \leq 2$, then $Z \leq 12 - X$ happens with probability 1 because $Z$ can only take values up to 10.
- If $2 < X \leq 10$, the probability is $\frac{12 - X}{10}$ because $Z$ needs to be less than $12 - X$, and this ratio follows from the uniform distribution of $Z$ over $[0, 10]$.

#### Step 3: Bayes Classifier
To minimize the probability of misclassification:
- If $\eta(x) \geq 0.5$, classify as $Y = 1$; otherwise, classify as $Y = 0$.
- From the calculated $\eta(x)$:
  - For $0 \leq x \leq 2$, $\eta(x) = 1$ (classify as $Y=1$).
  - For $2 < x \leq 10$, $\eta(x) = \frac{12-x}{10}$:
    - When $x = 7$, $\eta(7) = \frac{5}{10} = 0.5$.
    - For $x < 7$, $\eta(x) > 0.5$; hence classify as $Y=1$.
    - For $x > 7$, $\eta(x) < 0.5$; hence classify as $Y=0$.

#### Conclusion:
The Bayes classifier classifies a student as passing ($Y=1$) if $0 \leq x \leq 7$ and failing ($Y=0$) if $x > 7$. This decision rule minimizes the probability of misclassification based on the information available from $X$ alone.

To calculate the Bayes risk $R^*$ for the classification task, we begin by understanding the Bayes risk conceptually:

### Bayes Risk
The Bayes risk $R^*$ is the minimum expected probability of error for a classification system under the Bayesian framework. It's calculated using the expression:
$$
R^* = \mathbb{E}[\min\{\eta(X), 1 - \eta(X)\}],
$$
where $\eta(x)$ is the conditional probability of $Y=1$ given $X=x$, as derived in part (i). This expectation is effectively an integral over the possible values of $X$, weighted by their likelihood.

### Calculating $R^*$
Given that $X$ is uniformly distributed over $[0,10]$, each value of $X$ is equally likely, so the density function for $X$ is constant at $\frac{1}{10}$. The risk calculation then becomes an integral over the range of $X$, considering the minimum of $\eta(x)$ and $1 - \eta(x)$, the misclassification probabilities.

#### Recall:
From part (i), we derived:
$$
\eta(x) = 
\begin{cases} 
1 & \text{for } 0 \leq x \leq 2 \\
\frac{12 - x}{10} & \text{for } 2 < x \leq 10
\end{cases}
$$

#### Steps:
1. **Calculate $\min\{\eta(x), 1 - \eta(x)\}$ over the intervals of $x$**:
   - For $0 \leq x \leq 2$, $\eta(x) = 1$, hence $\min\{\eta(x), 1 - \eta(x)\} = 1 - 1 = 0$.
   - For $2 < x \leq 7$, $\eta(x) = \frac{12 - x}{10}$, and since $\eta(x) \geq 0.5$, $\min\{\eta(x), 1 - \eta(x)\} = 1 - \eta(x)$.
   - For $7 < x \leq 10$, $\eta(x) < 0.5$, hence $\min\{\eta(x), 1 - \eta(x)\} = \eta(x)$.

2. **Perform the Integration**:
$$
R^* = \int_2^7 \frac{1}{10} (1 - \eta(x)) dx + \int_7^{10} \frac{1}{10} \eta(x) dx
$$

   Expanding $\eta(x)$ in the integrals:
$$
\begin{aligned}
R^* & = \int_2^7 \frac{1}{10} \left(1 - \frac{12 - x}{10}\right) dx + \int_7^{10} \frac{1}{10} \left(\frac{12 - x}{10}\right) dx \\
& = \int_2^7 \frac{1}{10} \left(\frac{x - 2}{10}\right) dx + \int_7^{10} \frac{1}{10} \left(\frac{12 - x}{10}\right) dx
\end{aligned}
$$

   Simplifying these integrals and evaluating them:

#### For $\int_2^7 \frac{x - 2}{100} dx$:
$$
\frac{1}{100} \left[\frac{x^2}{2} - 2x \right]_2^7 = \frac{1}{100} \left[ \frac{49}{2} - 14 - (\frac{4}{2} - 4) \right] = \frac{1}{100} \left[ \frac{25}{2} \right] = \frac{25}{200}
$$

#### For $\int_7^{10} \frac{12 - x}{100} dx$:
$$
\frac{1}{100} \left[ 12x - \frac{x^2}{2} \right]_7^{10} = \frac{1}{100} \left[ 120 - 50 - (84 - 24.5) \right] = \frac{1}{100} \left[ 21 \right] = \frac{21}{200}
$$

Adding these two results gives the final Bayes risk:
$$
R^* = \frac{25}{200} + \frac{21}{200} = \frac{46}{200} = \frac{23}{100}.
$$

Therefore, the Bayes risk $R^*$, which quantifies the minimum expected probability of error, is $\frac{23}{100}$ or 23%. This represents the theoretical

 lowest error rate achievable by any classifier for this problem under the given conditions and assumptions.